In [15]:
import folium
import pandas as pd
import math
import random
import numpy as np
from haversine import haversine

In [16]:
def generate_coordinates_within_distance(latitude, longitude, distance):
    # Convert distance from meters to kilometers
    distance_km = random.uniform(0, distance) / 1000.0

    # Convert latitude and longitude to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)

    # Calculate a random bearing (in radians)
    random_bearing = random.uniform(0, 2 * math.pi)

    # Calculate the maximum angular distance (in radians) for generating coordinates
    max_angular_distance = distance_km / 6371.0

    # Calculate the new latitude
    new_lat_rad = math.asin(math.sin(lat_rad) * math.cos(max_angular_distance) + math.cos(lat_rad) * math.sin(max_angular_distance) * math.cos(random_bearing))

    # Calculate the new longitude
    new_lon_rad = lon_rad + math.atan2(math.sin(random_bearing) * math.sin(max_angular_distance) * math.cos(lat_rad), math.cos(max_angular_distance) - math.sin(lat_rad) * math.sin(new_lat_rad))

    # Convert the new latitude and longitude back to degrees
    new_latitude = math.degrees(new_lat_rad)
    new_longitude = math.degrees(new_lon_rad)

    return new_latitude, new_longitude

In [17]:
def generate_data(L, distance_meters, iterations):
    for given_latitude, given_longitude in L:
        for _ in range(iterations):
            generated_latitude, generated_longitude = generate_coordinates_within_distance(given_latitude, given_longitude, distance_meters)
            print(round(generated_latitude, 5), round(generated_longitude, 5), file=open("../../data/data_1.txt", "a"))

In [18]:
L = [(12.891442, 77.576690), (12.883189, 77.582958), (12.880897, 77.595722), (12.894222, 77.598295)]
distance_meters = 2000
iterations = 50

generate_data(L, distance_meters, iterations)

In [19]:
with open('../../data/data_1.txt', 'r') as file:
    L = []
    for line in file:
        if line and line != '\n':
            L.append([float(x) for x in line.strip().split(' ')])

In [20]:
column_names = ['latitude', 'longitude']
df = pd.DataFrame(L, columns=[column_names])
df.to_csv('../../data/data_1.csv', mode='a', index=False, header=False)

In [21]:
df = pd.read_csv('../../data/data_1.csv')
coordinates = df.values.tolist()

In [22]:
city_coords = (12.938655, 77.581057)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'black']
n = len(colors)

In [23]:
map = folium.Map(location=city_coords, zoom_start=15, control_scale = True)
for latitude, longitude in coordinates:
    folium.Marker(location=(latitude, longitude), icon=folium.Icon()).add_to(map)
map

In [24]:
def cluster_by_distance(points, radius):
    distances = []
    n = len(points)
    for i in range(n):
        distances.append([])
        for j in range(i + 1, n):
            distances[-1].append(haversine(points[i], points[j])) #unit is km

    # visited = [0]*n
    clusters = []
    for i in range(n):
        clusters.append([])
        count = 0
        for d in distances[i]:
            count += 1
            if d <= radius:
                clusters[-1].append(points[i+count])

    return clusters


In [25]:
radius, min_samples = 0.2, 15
clusters_1 = [x for x in cluster_by_distance(coordinates.copy(), radius) if len(x) >= min_samples]
map1 = folium.Map(location=city_coords, zoom_start=10)
for i in range(len(clusters_1)):
    cluster = clusters_1[i]
    color = colors[i % n]
    for point in cluster:
        x, y = point[0], point[1]
        folium.Marker(location= (x, y), icon=folium.Icon(color=color)).add_to(map1)
map


In [26]:
#https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

In [27]:
from sklearn.cluster import DBSCAN

In [28]:
epsilon = 0.5 / 6371.0
min_samples = 15
db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(df.to_numpy()))
labels = db.labels_
map2 = folium.Map(location=city_coords, zoom_start=15, control_scale = True)
for i in range(len(labels)):
    if labels[i] == -1:
        color = 'gray'
        folium.Marker(location= coordinates[i], icon=folium.Icon(color=color)).add_to(map2)

    else:
        color = colors[labels[i] % n]
        folium.Marker(location= coordinates[i], icon=folium.Icon(color=color)).add_to(map2)
map2